<a href="https://colab.research.google.com/github/zhiyuan-95/Amex-default-prediction-/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=JcAYmdwgUsuC7JZ7ELB69XlmZEDZ4a&prompt=consent&access_type=offline&code_challenge=KHtGu9J_7x3bm_iHrgazPLULU2C6_IVkpV7fS_rCDrw&code_challenge_method=S256

Enter authorization code: 4/0AZEOvhXTcPGwEA1z7jG2KK9WQeWldoH70fGM0FMQUT1EszzBlSKnp4PdEDyRjBN08_y30g

You are now logged in as [zhiyuan.jin1201@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID

In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
!gsutil cp gs://aedp/expanded_data/sample_train/part-00000 /content/sample_train.csv
!gsutil cp gs://aedp/expanded_data/sample_test/part-00000 /content/sample_test.csv

Copying gs://aedp/expanded_data/sample_train/part-00000...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

- [1 files][  4.8 GiB/  4.8 GiB]   26.0 MiB/s                                   
Operation completed over 1 objects/4.8 GiB.                                      
Copying gs://aedp/expanded_data/sample_test/part-00000...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

|
Operation completed over 1 objects/2.4 GiB.                                      


In [ ]:
import csv
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from kerastuner.tuners import RandomSearch

<ipython-input-4-89b5b0e1f941>:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
# Evaluation Method
def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    df = (pd.concat([y_true, y_pred], axis='columns')
          .sort_values('prediction', ascending=False))
    df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
    four_pct_cutoff = int(0.04 * df['weight'].sum())
    df['weight_cumsum'] = df['weight'].cumsum()
    df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
    return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    df = (pd.concat([y_true, y_pred], axis='columns')
          .sort_values('prediction', ascending=False))
    df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
    df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
    total_pos = (df['target'] * df['weight']).sum()
    df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
    df['lorentz'] = df['cum_pos_found'] / total_pos
    df['gini'] = (df['lorentz'] - df['random']) * df['weight']
    return df['gini'].sum()
def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    y_true_pred = y_true.rename(columns={'target': 'prediction'})
    return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)
    return 0.5 * (g + d)

In [ ]:
sample_train = 'sample_train.csv'
sample_test = 'sample_test.csv'
df_train = pd.read_csv(sample_train)
df_test = pd.read_csv(sample_test)
df_train.set_index('customer_ID', inplace = True)
df_test.set_index('customer_ID', inplace = True)

In [ ]:
def balanced_data(dataframe):
  target_1 = dataframe.loc[dataframe.target==1]
  target_0 = dataframe.loc[dataframe.target==0][:target_1.shape[0]]
  combine = pd.concat([target_1,target_0], axis = 0)
  balanced_dataframe = combine.iloc[np.random.permutation(combine.shape[0])]
  return balanced_dataframe

In [ ]:
cats = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
cat_top = []
for x in cats:
  cat_top.append(x+'_top')
num = list(df_train.columns)
for x in cat_top:
  num.remove(x)
def fill_na(data):
  data_cat = data[cat_top].astype(str).fillna('JIN')
  data_num = data[num].fillna(0)
  data_out = pd.concat([data_num,data_cat], axis = 1)
  return data_out

In [ ]:
#encode label to fit tensorflow
from sklearn.preprocessing import OneHotEncoder
def encoding(df,cat=cat_top):
  label = df[['target']]
  df = df.drop(columns = ['target'])
  encoder = OneHotEncoder(sparse_output = False)
  #encode label to fit tensorflow
  encoded_label = encoder.fit_transform(label)
  encoded_label_names = encoder.get_feature_names_out()
  label_encoded = pd.DataFrame(encoded_label, columns=encoded_label_names)
  label_encoded.index = df.index
  #encode categorical data
  encoded_columns = encoder.fit_transform(df[cat])
  encoded_column_names = encoder.get_feature_names_out()
  encoded_df = pd.DataFrame(encoded_columns, columns=encoded_column_names, index = df.index)
  feature_encoded = pd.concat([df.drop(columns=cat), encoded_df], axis=1)
  return stder(feature_encoded.astype(np.float64)), label_encoded

In [ ]:
from sklearn.preprocessing import StandardScaler
def stder(X):
    cols = X.columns
    stder = StandardScaler(copy=True, with_mean = True, with_std = True)
    stder.fit(X)
    return pd.DataFrame(stder.transform(X), columns = cols)

### Prepare training & test set

In [ ]:
# train
df_train = fill_na(df_train)
X_train_balanced = balanced_data(df_train)
X_train_encoded, y_train_encoded = encoding(X_train_balanced)
# Split data into train and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_encoded, y_train_encoded, test_size=0.3, random_state=42)

In [ ]:
# test
X_test = fill_na(df_test)
X_test,y_test = encoding(X_test)

### hyperparameter optimization

In [ ]:
import time
from tensorflow import keras
from keras import layers
import os
record_path = '/content/drive/MyDrive/Colab Notebooks/Default prediction'
os.makedirs(record_path, exist_ok=True)
dir = time.time()

In [ ]:
#X,y,X_val,y_val,d1,d2,lr = 1e-6, epks =5, loss_func='mean_squared_error',ac1 = 'elu', ac2 = 'sigmoid', bs =10
def model(hp):
    md = tf.keras.models.Sequential()
    md.add(layers.Flatten(input_shape=(1503,))),
    md.add(layers.BatchNormalization()),
    for i in range(n_layers):
      md.add(layers.Dense(hp.Int('input_units_{0}'.format(1+i),min_value = 200,max_value=3000,step = 50),activation=hp.Choice('activation_{0}'.format(i+1),['elu','sigmoid','tanh']))),
    md.add(layers.BatchNormalization()),

    md.add(layers.Dense(2, activation=hp.Choice('activation_end'.format(i),['sigmoid']))),
    lr = hp.Float("lr", min_value=1e-6, max_value=1e-5, sampling="log")

    md.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics = ['accuracy'],
                  loss='mean_squared_error')
    return md


In [ ]:
n_layers = 4
epks = 15
tuner = RandomSearch(
    model,
    objective = "val_loss",
    max_trials =200,
    executions_per_trial = 3,
    directory = record_path,
    overwrite = False,
    project_name = 'model_{0}l'.format(n_layers)
    )

In [ ]:
tuner.search(x=X_train,
             y=y_train,
             epochs=epks,
             batch_size=200,
             validation_data=(X_val,y_val))

In [ ]:
#tuner.results_summary()

In [ ]:
#tuner.get_best_models()[0].summary()

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0].values

In [ ]:
best_hp

{'input_units_1': 3500,
 'activation_1': 'elu',
 'input_units_2': 3100,
 'activation_2': 'sigmoid',
 'input_units_3': 200,
 'activation_3': 'tanh',
 'input_units_4': 2850,
 'activation_4': 'sigmoid',
 'activation_end': 'sigmoid',
 'lr': 3.4215203658712084e-06}

### best model

In [ ]:
#input_units_1,activation_1,input_units_2,activation2,input_units_3,activation_3

In [ ]:
def best_model(lr,activation_end,n_layers=4,**dense):
    md = tf.keras.models.Sequential()
    md.add(layers.Flatten(input_shape=(1503,))),
    md.add(layers.BatchNormalization()),
    for x in range(n_layers):
      md.add(layers.Dense(dense['input_units_{0}'.format(x+1)], activation=dense['activation_{0}'.format(x+1)]))
      md.add(layers.BatchNormalization())
    md.add(layers.Dense(2, activation=activation_end))
    md.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1.5e-5),
                  loss='mean_squared_error',metrics = ['accuracy'])
    md.fit(X_train,y_train,
           epochs = epks,
           batch_size=200,
           shuffle=True,
           validation_data = (X_val,y_val))
    return md

In [ ]:
def performance():
  model_trained = best_model(**best_hp)
  prediction = model_trained.predict(X_test)[:,1]
  target_test = y_test[['target_1']].rename(columns={'target_1': 'target'})
  prediction = pd.DataFrame(prediction,columns = ['prediction'])
  prediction.index = target_test.index
  return (amex_metric(target_test,prediction))

In [ ]:
performance()

Epoch 1/15
414/414 [==============================] - 8s 10ms/step - loss: 0.0958 - accuracy: 0.8751 - val_loss: 0.0870 - val_accuracy: 0.8845
Epoch 2/15
414/414 [==============================] - 3s 8ms/step - loss: 0.0832 - accuracy: 0.8906 - val_loss: 0.0863 - val_accuracy: 0.8843
Epoch 3/15
414/414 [==============================] - 3s 8ms/step - loss: 0.0770 - accuracy: 0.8996 - val_loss: 0.0859 - val_accuracy: 0.8851
Epoch 4/15
414/414 [==============================] - 3s 8ms/step - loss: 0.0717 - accuracy: 0.9079 - val_loss: 0.0864 - val_accuracy: 0.8849
Epoch 5/15
414/414 [==============================] - 3s 8ms/step - loss: 0.0672 - accuracy: 0.9165 - val_loss: 0.0860 - val_accuracy: 0.8843
Epoch 6/15
414/414 [==============================] - 3s 8ms/step - loss: 0.0633 - accuracy: 0.9227 - val_loss: 0.0847 - val_accuracy: 0.8869
Epoch 7/15
414/414 [==============================] - 3s 8ms/step - loss: 0.0594 - accuracy: 0.9293 - val_loss: 0.0861 - val_accuracy: 0.8844
Epoch

0.713730489137953

## Prediction

In [ ]:
!gsutil cp gs://aedp/expanded_data/test_expansion/part-00000 /content/test.csv

In [ ]:
test = 'test.csv'
df_test = pd.read_csv(test)
df_test.set_index('customer_ID', inplace = True)

In [ ]:
def test_encoding(df,cat=cat_top):
  encoder = OneHotEncoder(sparse_output = False)
  encoded_columns = encoder.fit_transform(df[cat])
  encoded_column_names = encoder.get_feature_names_out()
  encoded_df = pd.DataFrame(encoded_columns, columns=encoded_column_names, index = df.index)
  feature_encoded = pd.concat([df.drop(columns=cat), encoded_df], axis=1)
  return stder(feature_encoded.astype(np.float64))

In [ ]:
df_test = fill_na(df_test)
test_set = encoding(X_test)

In [ ]:
model_trained = best_model(**best_hp)
prediction = model_trained.predict(X_test)[:,1]
target_test = y_test[['target_1']].rename(columns={'target_1': 'target'})
prediction = pd.DataFrame(prediction,columns = ['prediction'])
prediction.index = target_test.index